In [12]:

import requests
import os
import json
from openai import OpenAI
from bs4 import BeautifulSoup
import re
from typing import List, Dict
import time
import pandas as pd

In [13]:
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_API")
client = OpenAI()

In [14]:
url = "https://terms.naver.com/list.naver?cid=44630&categoryId=44630"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [15]:
# LLM 호출 함수 (OpenAI API 예시)
def food_name_from_title(title: str) -> str:

    try:
        # LLM 호출
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "주어진 데이터는 제목이야. 여기서 요리의 이름만 뽑아줘. 요리의 제목은 1개야."},
                {
                    "role": "user",
                    "content": title
                }
            ]
        )
        recipe_name = completion.choices[0].message.content
        return recipe_name
    
    except Exception as e:
        print(f"LLM 호출 에러: {e}")
        return "요리 이름 추출 실패"
    

In [16]:
# def get_recipt_detail1(url: str) -> dict:

#     def parse_basic_info_from_html(soup) -> dict:
#         basic_info = {}
#         info_section = soup.find("h4", id="TABLE_OF_CONTENT3")
#         if info_section:
#             info_paragraph = info_section.find_next("p")
#             if info_paragraph:
#                 for strong_tag in info_paragraph.find_all("strong"):
#                     key = strong_tag.text.strip("·").strip(" :").strip()
#                     value = strong_tag.next_sibling.strip() if strong_tag.next_sibling else ""
#                     basic_info[key] = value
#         return basic_info

#     try:
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, "html.parser")

#         recipt_data = {}

#         # 기본정보 추출
#         recipt_data["basic_info"] = parse_basic_info_from_html(soup)

#         return recipt_data

#     except requests.exceptions.RequestException as e:
#         print(f"요청 중 에러 발생: {e}")
#         return {"error": "요청 실패"}
    
# url = "https://terms.naver.com/entry.naver?docId=1988772"  # 예시 URL
# data = get_recipt_detail1(url)
# print(data["basic_info"])


In [17]:
def get_recipt_list(page : int) -> str:

    params = {"page": page, "view_type" : "sm"}
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        return response.text
        
    
    except requests.exceptions.RequestException as e:
        print(f"{page} 에러발생 {e}")
        return ""


In [18]:
def get_recipt_detail(url : str) -> Dict:
    
    recipt_data = {
        "요리재료" : [],
        "기본정보" : {},
        "조리순서" : []
    }
    
    # 페이지에서 내용 가져오기
    try :
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 요리 재료 추출
        ingredient_h4 = soup.find("h4", id="TABLE_OF_CONTENT2")
        if ingredient_h4:
            ingredient_text = ingredient_h4.find_next("p").text
            ingredients_clean = re.sub(r'· (주재료) : ', '', ingredient_text)
            ingredients_clean = ingredients_clean.replace("· 부재료 :" , ",")
            ingredients_clean = ingredients_clean.replace("\xa0" , " ")
            
            ingreidents_list = [item.strip() for item in ingredients_clean.split(",")]
            
            for item in ingreidents_list:
                parts = item.rsplit(" ",1)
                if len(parts) ==2:
                    name, amount = parts
                    recipt_data['요리재료'].append({"재료":name.strip(), "용량":amount.strip()})
                
                
        
 
        # 기본정보 추출
        info_h4 = soup.find("h4", id="TABLE_OF_CONTENT3")
        if info_h4:  
            info_para = info_h4.find_next("p")
            if info_para:
                for strong_tag in info_para.find_all("strong"):
                    key = strong_tag.text.strip("·").strip(" :").strip()
                    value = strong_tag.next_sibling.strip() if strong_tag.next_sibling else ""
                    recipt_data["기본정보"][key] = value

        
        
        #요리과정 추출
        recipt_h4 = soup.find("h4",id="TABLE_OF_CONTENT4")
        recipt_list = []
        #과정 순회
        if recipt_h4:
            for tag in recipt_h4.find_all_next():
                if tag.name =="h3" or tag.name =="h4":
                    break
                if tag.name == "p" and tag.text.strip():
                    recipt_list.append(tag.text.strip())
        recipt_data['조리순서'] = recipt_list
        return recipt_data
    
      
    except requests.exceptions.RequestException as e:
        print(f"기사 내용 요청 중 에러 발생: {e}")
        return {"error":"요청 실패"}

In [19]:
def parse_recipt_info(html: str) -> List[Dict]:
    recipts = []
    soup = BeautifulSoup(html,"html.parser")
    
    recipts_list = soup.find("ul", class_="content_list")
    if not recipts_list:
        return recipts
    
    new_recipts = recipts_list.find_all("li", recursive=False)
    print(f"{len(new_recipts)}개의 레시피를 찾았습니다.")
    
    for item in new_recipts:
        
        
        a_tag = item.find("a", href=True, onclick=True)
        
        if not a_tag:
            print("cant find a tag")
            continue
        
          
        title = a_tag.text.strip() if a_tag else ""
        print(title)
        
        food_name = food_name_from_title(title)
        
        

        url_elem = a_tag["href"] if a_tag else ""
        url = "https://terms.naver.com" + url_elem
        print(url)
        
        recipt = get_recipt_detail(url) if url else ""
        
        food_recipt = {
            "요리명" : food_name,
            "요리재료" : recipt["요리재료"],
            "기본정보" : recipt["기본정보"],
            "조리순서" : recipt["조리순서"]
        }
        
        # food_name = ~~[1]
        recipts.append(food_recipt)
        
        time.sleep(1)
    
    
    return recipts
    
        

In [20]:

def save_to_files(recipts: List[Dict], start_page: int, end_page: int, base_path: str = "./"):

    # DataFrame 생성
    df = pd.DataFrame(recipts)
    
    
    file_name = f"ai_times_news_{start_page}-{end_page}.json"
    
    
    # DataFrame을 JSON 파일로 저장
    json_path = os.path.join(base_path, file_name)
    df.to_json(json_path, force_ascii=False, orient='records', indent=4)
    print(f"JSON 파일 저장 완료: {json_path}")
    
    # 데이터 수집 정보 출력
    print("\n[데이터 수집 정보]")
    print(f"수집 페이지: {start_page}-{end_page} 페이지")
    print(f"수집 레시피: 총 {len(df)}건")
    
    # 데이터 미리보기
    print("\n[데이터 미리보기]")
    print(df.head())

In [21]:
def main():
    start_page = 1
    end_page = 1 # 수집할 마지막 페이지 번호
    page_unit = 10# 한 번에 처리할 페이지 수
    download_folder = './food_recipts' #저장 폴더 이름
    os.makedirs(download_folder, exist_ok=True) #폴더 생성
    print("레시피 정보 수집 중...")
    
    # 10페이지씩 처리
    for start_idx in range(start_page, end_page + 1, page_unit):
        all_articles = []
        end_idx = min(start_idx + page_unit - 1, end_page)
        
        print(f"\n=== {start_idx}~{end_idx} 페이지 수집 시작 ===")
        
        for page in range(start_idx, end_idx + 1):
            html = get_recipt_list(page)
            if html:
                page_articles = parse_recipt_info(html)
                all_articles.extend(page_articles)
                print(f"페이지 {page}: {len(page_articles)}개의 레시피 정보 수집 완료")
        
        if not all_articles:
            print(f"{start_idx}~{end_idx} 페이지의 음식 레시피 정보를 가져오는데 실패했습니다.")
            continue
        
        print(f"\n{start_idx}~{end_idx} 페이지 : 총 {len(all_articles)}개의 레시피 정보를 수집했습니다.")
        
        try:
            save_to_files(all_articles, start_idx, end_idx, download_folder)
        except Exception as e:
            print(f"데이터 저장 중 에러 발생: {e}")
        
        print(f"=== {start_idx}~{end_idx} 페이지 처리 완료 ===\n")
        
        # 다음 수집 전 잠시 대기
        time.sleep(0.1)

    print("\n모든 페이지 수집 완료!")

if __name__ == "__main__":
    main()

레시피 정보 수집 중...

=== 1~1 페이지 수집 시작 ===
15개의 레시피를 찾았습니다.

https://terms.naver.com/entry.naver?docId=1988772&cid=48173&categoryId=48173

https://terms.naver.com/entry.naver?docId=1988548&cid=48161&categoryId=48197

https://terms.naver.com/entry.naver?docId=1988751&cid=48173&categoryId=48173

https://terms.naver.com/entry.naver?docId=3390165&cid=48183&categoryId=48278

https://terms.naver.com/entry.naver?docId=1993248&cid=48180&categoryId=48249

https://terms.naver.com/entry.naver?docId=1988798&cid=48173&categoryId=48173

https://terms.naver.com/entry.naver?docId=3390177&cid=48183&categoryId=48278

https://terms.naver.com/entry.naver?docId=3390183&cid=48183&categoryId=48278

https://terms.naver.com/entry.naver?docId=1988409&cid=48173&categoryId=48173

https://terms.naver.com/entry.naver?docId=1988882&cid=48173&categoryId=48173

https://terms.naver.com/entry.naver?docId=1721758&cid=48180&categoryId=48256

https://terms.naver.com/entry.naver?docId=3390245&cid=48183&categoryId=48278

https://

In [22]:
food = get_recipt_detail(url)
food

{'요리재료': [], '기본정보': {}, '조리순서': []}